In [34]:
import pandas as pd
import numpy as np
from PIL import Image
import os

Preprocessing(Dont Execute)

In [ ]:

csv_path = "handwriting_data.csv"
data = pd.read_csv(csv_path)

processed_data = []

image_size = (28, 28)

for index, row in data.iterrows():
    image_path = row['path']
    label = row['label']
    
    try:
        img = Image.open(image_path).convert('L')
        img_resized = img.resize(image_size)
        pixel_values = np.array(img_resized).flatten()
        processed_data.append([label] + pixel_values.tolist())
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

columns = ['label'] + [f'pixel_{i}' for i in range(image_size[0] * image_size[1])]
processed_df = pd.DataFrame(processed_data, columns=columns)


In [ ]:
processed_df.head()

In [ ]:
processed_df.to_csv("./dataset_processed.csv")

Model Execution from here

In [51]:
from tensorflow.keras import layers, models

def create_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

input_shape = (28, 28, 1)
num_classes = 26 

model = create_model(input_shape, num_classes)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

C:\Users\Harsh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 3, 3, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 26)                  │           3,354 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 132,954 (519.35 KB)

 Trainable params: 132,954 (519.35 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
import pandas as pd
import numpy as np
from PIL import Image
import os

csv_path = "dataset_processed.csv"
df = pd.read_csv(csv_path)

labels = df['label']
pixels = df.drop('label', axis=1)

In [53]:
pixels.shape

(17376, 784)

In [54]:
X = pixels.values.reshape(-1, 28, 28, 1)
X = X / 255.0 

In [55]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(labels)
y_encoded

array([ 0,  0,  0, ..., 25, 25, 25])

In [56]:
y = y_encoded

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [58]:
history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1674 - loss: 2.8101 - val_accuracy: 0.3619 - val_loss: 2.0333
Epoch 2/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3534 - loss: 2.0822 - val_accuracy: 0.5282 - val_loss: 1.5554
Epoch 3/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.4862 - loss: 1.6533 - val_accuracy: 0.6171 - val_loss: 1.2806
Epoch 4/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5713 - loss: 1.3557 - val_accuracy: 0.6680 - val_loss: 1.0759
Epoch 5/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6274 - loss: 1.1686 - val_accuracy: 0.6752 - val_loss: 1.0112
Epoch 6/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.6753 - loss: 1.0194 - val_accuracy: 0.7301 - val_loss: 0.8746
Epoch 7/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6929 - loss: 0.9593 - val_accuracy: 0.7428 - val_loss: 0.8239
Epoch 8/40
435/435 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.7279 - loss: 0.8488 - val_accuracy: 0

In [42]:
img_path = "t.jpg"
img = Image.open(img_path).convert('L')
img_resized = img.resize((28, 28))
img_array = np.array(img_resized) / 255.0  

img_reshaped = img_array.reshape(1, 28, 28, 1)

predicted_class = model.predict(img_reshaped)

predicted_label = np.argmax(predicted_class)

predicted_char = label_encoder.inverse_transform([predicted_label])

print(f"Predicted character: {predicted_char[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted character: U


In [17]:
import requests

API_KEY = "pysBFXCh_vbaxZtyGCppB_EkWNL3e95M0x0mLBPHZk2D"
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":
 API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

In [45]:
img_path = "t.jpg"
img = Image.open(img_path).convert('L')
img_resized = img.resize((28, 28))
img_array = np.array(img_resized) / 255.0  
img_reshaped = img_array.reshape(28, 28, 1).tolist()

payload_scoring = {
    "input_data": [
        {
            "fields": ["pixels"], 
            "values": [img_reshaped]
        }
    ]
}


response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/6ea6801d-9fc9-495c-ab2d-64fbd6492f10/predictions?version=2021-05-01', json=payload_scoring,
 headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())



Scoring response
{'predictions': [{'id': 'dense_1', 'values': [[2.1957178830689372e-07, 2.2302608317683855e-15, 2.0428220523172058e-05, 9.185914806586482e-10, 1.828929453040473e-05, 2.0741440494020935e-06, 1.186183098411675e-07, 8.667440098975021e-10, 7.870451794629585e-13, 0.0007809540838934481, 3.153111904907746e-08, 6.980367470532656e-05, 1.846906449998187e-08, 0.0074252355843782425, 9.732854950315114e-15, 1.4796098790625933e-09, 4.5018444570361476e-17, 1.1745446837352347e-09, 3.5854930136025587e-11, 1.9089985148212918e-10, 0.9916184544563293, 6.418839620891958e-05, 6.754108383688617e-11, 5.970633601241893e-13, 1.8868399820348714e-07, 7.350232067437901e-08]]}]}


In [47]:
response_json = response_scoring.json()

probabilities = response_json['predictions'][0]['values'][0]

predicted_class = np.argmax(probabilities)

print(f"Predicted Class: {predicted_class}")

predicted_char = label_encoder.inverse_transform([predicted_class])
predicted_char[0]

Predicted Class: 20


'U'

In [48]:
from ibm_watson import TextToSpeechV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('QJM2EY7bhf2M15MloYBDhyny3TcWJRkNJ_xfM0MuxhVV')
text_to_speech = TextToSpeechV1(
    authenticator=authenticator
)

text_to_speech.set_service_url('https://api.au-syd.text-to-speech.watson.cloud.ibm.com/instances/218d8ce1-fd5f-4a54-b1f6-8ab82922458b')

with open('hello_world.wav', 'wb') as audio_file:
    audio_file.write(
        text_to_speech.synthesize(
            predicted_char[0],
            voice='en-US_AllisonV3Voice',
            accept='audio/wav'        
        ).get_result().content)

In [50]:
pronunciation = text_to_speech.get_pronunciation(
    text='IEEE',
    voice='en-US_AllisonV3Voice',
    format='ibm'
).get_result()
print(pronunciation)

{'pronunciation': '`[.1Y] `[.1i] `[.1i] `[.1i]'}


In [ ]:
from os.path import join, dirname
import json
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

speech_to_text.set_service_url('https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/93b0774b-3417-4097-a169-0196d324c2e9')

with open(join(dirname(__file__), './.', 'hello_world.wav'),
               'rb') as audio_file:
    speech_recognition_results = speech_to_text.recognize(
        audio=audio_file,
        content_type='audio/wav',
        word_alternatives_threshold=0.9,
    ).get_result()
print(json.dumps(speech_recognition_results))